# 自定义层

深度学习成功背后的一个因素是神经网络的灵活性：
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。
在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。

## 不带参数的层

首先，我们(**构造一个没有任何参数的自定义层**)。
回忆一下在 :numref:`sec_model_construction`对块的介绍，
这应该看起来很眼熟。
下面的`CenteredLayer`类要从其输入中减去均值。
要构建它，我们只需继承基础层类并实现前向传播功能。


In [1]:
import torch
import torch.nn.functional as F
from torch import nn

# 这是一个自定义的CenteredLayer类，继承自nn.Module。这个层的作用是将输入张量减去其均值。
class CenteredLayer(nn.Module):
    # 在__init__方法中，我们调用了父类的构造函数super().__init__()来初始化nn.Module的基础功能。
    def __init__(self):
        super().__init__()

    # 在forward方法中，我们定义了层的前向传播操作。给定输入张量X，我们计算其均值，并将每个元素减去均值。然后返回结果张量。
    def forward(self, X):
        return X - X.mean()

让我们向该层提供一些数据，验证它是否能按预期工作。


In [2]:
# 如果你将创建的CenteredLayer实例命名为layer，可以通过调用layer()方法来使用该层。在这种情况下，你可以将一个张量作为输入传递给该层。请注意，输入张量应该是浮点类型的张量。

# 例如，如果你传递一个包含数字1、2、3、4、5的张量，你将得到一个以0为中心的张量。运行以下代码可以获得结果：
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在，我们可以[**将层作为组件合并到更复杂的模型中**]。


In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(9.3132e-10, grad_fn=<MeanBackward0>)

## [**带参数的层**]

以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数。


In [5]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来，我们实例化`MyLinear`类并访问其模型参数。


In [6]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.2442, -0.0230, -0.8793],
        [ 0.1244,  0.3421, -0.1684],
        [ 0.1678,  0.3329, -0.7883],
        [-0.6503, -0.3536,  0.7957],
        [ 0.8285, -0.8747, -1.9052]], requires_grad=True)

我们可以[**使用自定义层直接执行前向传播计算**]。


In [7]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.7173, 0.0000],
        [0.2918, 0.1684, 0.0000]])

我们还可以(**使用自定义层构建模型**)，就像使用内置的全连接层一样使用自定义层。


In [8]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
1. 设计一个返回输入数据的傅立叶系数前半部分的层。


[Discussions](https://discuss.d2l.ai/t/1835)
